<a href="https://colab.research.google.com/github/RiskyTrick/Internship-Projects/blob/master/Intrusion_Detection_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'network-intrusion-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3674161%2F6376134%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240404%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240404T031152Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D872d66d19c3ce1bf2a0a6d6d292b2d5dcf2ad4b601213152d38dd59af7cb0de782931ea8491772ad8e63fa2d2c9c843306c68d185e81291b2a5176a304b9b29396c904bda03e47480a138205cc197cfd9c7222ef47155a4fd006fdfdb946b1cc92a9fc2f0297c960f76294e3f3881cc0676275a45a079196d4ec2f870a1fce39081be3c5da7f3b27d13437b4eaa2c05f0bc5d3ccda6035595f6a0f3b86be9e88286f126a2c12f841ac277477917a86bcd645647ad7e0c40c904c0a74326ebbd26132b850365a4a8b407a1dc52edb7f3319a60328c832fdde16071231d11afc08687e3291c8bf955f76c2b09b44d17a61421ffdf813e00822d5e4b43742fe3cb9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[=====================================             ] 179200000 bytes downloaded

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/kaggle/input/network-intrusion-dataset/Wednesday-workingHours.pcap_ISCX.csv')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df

In [ ]:

from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()
df[' Label']=encoder.fit_transform(df[' Label'])

In [ ]:
df.head()

# here 0 = benign ,1 = dos golden eye,2= dos hulk ,3= dos slowhttptest

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.model_selection import train_test_split

In [ ]:
Target= ' Label'
X = df.loc[:, df.columns != Target]
y = df.loc[:,Target]

In [ ]:
from sklearn.impute import SimpleImputer
lower_bound=0
upper_bound=100
X_clipped = np.clip(X, lower_bound, upper_bound)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_clipped)

In [ ]:
selector = SelectKBest(score_func=chi2, k=10)
selector.fit(X_imputed, y)
selected_features = selector.get_support(indices=True)
X_new = X_imputed[:, selected_features]
X_new_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.25)

In [ ]:
print(selected_features)

In [ ]:
X_new

In [ ]:
y.unique()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor()
dtr.fit(X_new_train,y_train)

In [ ]:
y_pred=dtr.predict(X_test)

In [ ]:
y_pred[:10]

In [ ]:
y_test[:10]

In [ ]:
from sklearn.metrics import mean_squared_error


In [ ]:
mean_squared_error( y_test,y_pred)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_reg=RandomForestRegressor(n_estimators=500 )
rf_reg.fit(X_new_train,y_train)

In [ ]:
y_pred=rf_reg.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred = pd.DataFrame(data=y_pred,columns=['pred'])
y_pred

In [ ]:
y_test

In [ ]:
print(mean_squared_error(y_test,y_pred))